Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat.
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат.

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

In [4]:
data = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True)
data.dropna(inplace=True)
print(data.info())
data.head()

C:\Users\d.kam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 2 to 1021965
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                396634 non-null  object 
 1   user_id           396634 non-null  float64
 2   venue_id          396634 non-null  float64
 3   latitude          396634 non-null  float64
 4   longitude         396634 non-null  float64
 5   created_at        396634 non-null  object 
dtypes: float64(4), object(2)
memory usage: 21.2+ MB
None


,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Будем использовать алгоритм MeanShift, bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

На 396634 строках кластеризация будет работать долго. Возьмем сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.


In [12]:
data_cl = data.values[0:100000, 3:5]
print('Shape: ' + str(data_cl.shape))
print(data_cl)

Shape: (100000, 2)
[[38.8951118 -77.0363658]
 [33.800745 -84.41052]
 [45.5234515 -122.6762071]
 ...
 [29.7628844 -95.3830615]
 [32.802955 -96.769923]
 [37.7749295 -122.4194155]]


In [13]:
cluster = MeanShift(bandwidth=0.1)
cluster.fit(data_cl)

MeanShift(bandwidth=0.1)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [58]:
labels = cluster.labels_
cluster_centers = cluster.cluster_centers_
labels_unique, labels_counts = np.unique(labels, return_counts=True)

In [59]:
labels_unique_more = []
for index, cnt in enumerate(labels_counts):
    if cnt > 15:
        labels_unique_more.append(labels_unique[index])
print(len(labels_unique_more))

592


In [42]:
cluster_centers_new = np.empty( (len(labels_unique_more), 2) )
for index, lbl in enumerate(labels_unique_more):
    cluster_centers_new[index] = cluster_centers[lbl]

print(cluster_centers_new)

[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  41.61853175  -88.44556818]
 [  39.2494686   -77.1821271 ]
 [  38.65877915  -76.8856871 ]]


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

In [43]:
offices = pd.read_csv('Offices.csv', skipinitialspace=True)
print(offices)

    latitude   longitude
0  33.751277 -118.188740
1  25.867736  -80.324116
2  51.503016   -0.075479
3  52.378894    4.885084
4  39.366487  117.036146
5 -33.868457  151.205134


Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [44]:
def distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

In [55]:
distance_office = np.empty(cluster_centers_new.shape[0])
for i, item in enumerate(cluster_centers_new):
    min_dist = distance(item, offices.loc[0])
    for j in range(len(offices)):
        dist = distance(item, offices.loc[j])
        if dist < min_dist:
            min_dist = dist
    distance_office[i] = min_dist

In [56]:
dist_sorted_indices = np.argsort(distance_office)
distance_office_sorted = distance_office[dist_sorted_indices]
cluster_centers_new_sorted = cluster_centers_new[dist_sorted_indices]

In [57]:
print(distance_office_sorted[0])
print(str(cluster_centers_new_sorted[0]))

0.007834758163107856
[-33.86063043 151.20477593]
